# Link Prediction on Heterogenous Graphs
##### Web Based Systems 2025 @ [FCSE](https://finki.ukim.mk/)

The topic of this exercise is link prediction on a graph generated from the extracted entities and relations from L2 - Knowledge Graph Construction.

## Requirements

In [1]:
import pandas as pd
import torch
from torch_geometric.data import HeteroData
import torch_geometric.transforms as T

## Link Prediction on Stock News Headlines

We'll load a  `*.csv` file as input and construct a heterogeneous graph from it. Then, we'll use this dataset as input into a [heterogeneous graph model](https://pytorch-geometric.readthedocs.io/en/latest/notes/heterogeneous.html#hgtutorial), and use it for the task of link prediction.

We are going to use the triples extracted from the Stock News Headlines in AV 07 - Knowledge Graph Construction.

Note that we do not have many entities and relations in the dataset, since we extracted triples only from a subset of news headlines. Nevertheless, this data is enought to illustrate how to perform link prediction on heterogenous graphs.

The link prediction task then tries to predict missing industries for organizations mentioned in the stock news headlines, which is a usefull information for further tasks such as stock price prediction.

### Heterogeneous Graph Creation

First, we will import the dataset using `pandas`.

In [2]:
df = pd.read_csv("kg_stock_headlines_normalized.csv")

In [3]:
df.head()

,head,head_type,type,tail,tail_type
0,Gran,org,country,Russia,loc
1,Russia,loc,item operated,Gran,org
2,Russia,loc,subsidiary,Gran,org
3,2010,time,point in time,2010,date
4,third quarter of 2010,time,point in time,2010,date


Let's see what types of entities and relations we have.

In [4]:
df['head_type'].unique()

array(['org', 'loc', 'time', 'concept', 'eve', 'media', 'per', 'misc'],
      dtype=object)

In [5]:
df['tail_type'].unique()

array(['loc', 'org', 'date', 'concept', 'eve', 'media', 'misc', 'per',
       'num', 'time'], dtype=object)

We will only use head, relation, tail triple types for which we have at least 100 extracted triples.

Note: Sometimes the extracted triples can have errors, so it's a good idea to inspect them before including them in the model, and remove any wrong information.

Before we create the heterogeneous graph, let’s take a look at the data.

In [6]:
df[['head_type', 'type', 'tail_type']].value_counts()

head_type  type                                                  tail_type
org        country                                               loc          260
concept    subclass of                                           concept      183
org        industry                                              concept      122
           subsidiary                                            org          102
concept    facet of                                              concept       92
                                                                             ... 
per        substitute/deputy/replacement of office/officeholder  per            1
time       follows                                               concept        1
per        work period (start)                                   date           1
time       organizer                                             org            1
           start time                                            date           1
Name: count, Length: 24

We can see that we have more than 100 triples only for the first 4 triple types, namely:
 * org -> country -> loc
 * concept -> subclass of -> concept
 * org -> industry -> concept
 * org -> subsidiary -> org

Let's take the top 4 types of triples to include in the graph.

In [7]:
relations = list(df[['head_type', 'type', 'tail_type']].value_counts().head(4).index)

In [8]:
relations

[('org', 'country', 'loc'),
 ('concept', 'subclass of', 'concept'),
 ('org', 'industry', 'concept'),
 ('org', 'subsidiary', 'org')]

Here, we first create a mapping that maps entry IDs to a consecutive value in the range `{ 0, ..., num_rows - 1 }`.
This is needed as we want our final data representation to be as compact as possible, *e.g.*, the representation of an organization in the first row should be accessible via `x[0]`.

Afterwards, we obtain the final `edge_index` representation of shape `[2, num_edges_of_type]` for each different type of edge.

For each triple type combination we store the nodes in a dictionary based on their type.

Let's create the edges in the COO PyTorch format.
Reminder:
 - The format looks like this: [[0, 0], [1, 2]]

The example above can be read as node 0 is connected to node 1, and node 0 is connceted to node 2.

We create a dictionary of COO indices for each head-relation-tail type.

In [9]:
id_to_node_mapping = {}

In [10]:
relations_by_type = {}

In [11]:
def get_or_set_id_node_mapping(entity, entity_type, mapping_dict):
  if entity_type not in mapping_dict:
    mapping_dict[entity_type] = {}

  id_entity = mapping_dict[entity_type].get(entity, -1)
  if id_entity == -1:
    id_entity = len(mapping_dict[entity_type])
    mapping_dict[entity_type][entity] = id_entity

  return id_entity

In [12]:
for i, row in df.iterrows():
  head = row['head']
  head_type = row['head_type']
  rel = row['type']
  tail = row['tail']
  tail_type = row['tail_type']

  if (head_type, rel, tail_type) not in relations:
    continue


  id_head = get_or_set_id_node_mapping(head, head_type, id_to_node_mapping)
  id_tail = get_or_set_id_node_mapping(tail, tail_type, id_to_node_mapping)

  rel_type = (head_type, rel, tail_type)

  if rel_type not in relations_by_type:
    relations_by_type[rel_type] = [[], []]

  relations_by_type[rel_type][0].append(id_head)
  relations_by_type[rel_type][1].append(id_tail)

In [13]:
id_to_node_mapping.keys()

dict_keys(['org', 'loc', 'concept'])

In [14]:
relations_by_type.keys()

dict_keys([('org', 'country', 'loc'), ('org', 'industry', 'concept'), ('concept', 'subclass of', 'concept'), ('org', 'subsidiary', 'org')])

With this, we are ready to initialize our `HeteroData` object and pass the necessary information to it.
Note that we also pass in a `node_id` vector to each node type in order to reconstruct the original node indices from sampled subgraphs.
We also take care of adding reverse edges to the `HeteroData` object.
This allows our GNN model to use both directions of the edge for message passing:

In [15]:
data = HeteroData()

for entity_type in id_to_node_mapping.keys():
  data[entity_type].node_id = torch.arange(len(id_to_node_mapping[entity_type]))

for rel_type in relations_by_type.keys():
  data[rel_type].edge_index = torch.tensor(relations_by_type[rel_type])

# We also need to make sure to add the reverse edges from movies to users
# in order to let a GNN be able to pass messages in both directions.
# We can leverage the `T.ToUndirected()` transform for this from PyG:
data = T.ToUndirected()(data)

In [16]:
data

HeteroData(
  org={ node_id=[385] },
  loc={ node_id=[35] },
  concept={ node_id=[361] },
  (org, country, loc)={ edge_index=[2, 260] },
  (org, industry, concept)={ edge_index=[2, 122] },
  (concept, subclass of, concept)={ edge_index=[2, 352] },
  (org, subsidiary, org)={ edge_index=[2, 182] },
  (loc, rev_country, org)={ edge_index=[2, 260] },
  (concept, rev_industry, org)={ edge_index=[2, 122] }
)

We have 3 entity types:
 - 'org' - 385 nodes
 - 'loc' - 35 nodes
 - 'concept' - 361 nodes

 and 6 edge types:
 - ('org', 'country', 'loc') - 260 edges
 - (org, industry, concept) -  122 edges
 - (concept, subclass of, concept) - 352 edges
 - (org, subsidiary, org) - 182 edges


and the reverse edges


### Defining Edge-Level Training Splits

Since our data is now ready-to-be-used, we can split the ratings of users into training, validation, and test splits.
This is needed in order to ensure that we leak no information about edges used during evaluation into the training phase.
For this, we make use of the [`transforms.RandomLinkSplit`](https://pytorch-geometric.readthedocs.io/en/latest/modules/transforms.html#torch_geometric.transforms.RandomLinkSplit) transformation from PyG.
This transformation randomly divides the edges in the `("org", "industry", "concept")` into training, validation and test edges.
The `disjoint_train_ratio` parameter further separates edges in the training split into edges used for message passing (`edge_index`) and edges used for supervision (`edge_label_index`).
Note that we also need to specify the reverse edge type `("concept", "rev_industry", "org")`.
This allows the `RandomLinkSplit` transform to drop reverse edges accordingly to not leak any information into the training phase.

In [17]:
target_edge_type = ("org", "industry", "concept")
target_edge_rev_type = ("concept", "rev_industry", "org")

In [18]:
# For this, we first split the set of edges into
# training (80%), validation (10%), and testing edges (10%).
# Across the training edges, we use 80% of edges for message passing,
# and 20% of edges for supervision.
# We further want to generate fixed negative edges for evaluation with a ratio of 2:1.
# Negative edges during training will be generated on-the-fly.
# We can leverage the `RandomLinkSplit()` transform for this from PyG:
transform = T.RandomLinkSplit(
    num_val=0.1,
    num_test=0.1,
    disjoint_train_ratio=0.8,
    neg_sampling_ratio=2.0,
    add_negative_train_samples=False,
    edge_types=target_edge_type,
    rev_edge_types=target_edge_rev_type,
)

train_data, val_data, test_data = transform(data)
print("Training data:")
print("==============")
print(train_data)
print()
print("Validation data:")
print("================")
print(val_data)
print("Test data:")
print("================")
print(test_data)

Training data:
HeteroData(
  org={ node_id=[385] },
  loc={ node_id=[35] },
  concept={ node_id=[361] },
  (org, country, loc)={ edge_index=[2, 260] },
  (org, industry, concept)={
    edge_index=[2, 20],
    edge_label=[78],
    edge_label_index=[2, 78],
  },
  (concept, subclass of, concept)={ edge_index=[2, 352] },
  (org, subsidiary, org)={ edge_index=[2, 182] },
  (loc, rev_country, org)={ edge_index=[2, 260] },
  (concept, rev_industry, org)={ edge_index=[2, 20] }
)

Validation data:
HeteroData(
  org={ node_id=[385] },
  loc={ node_id=[35] },
  concept={ node_id=[361] },
  (org, country, loc)={ edge_index=[2, 260] },
  (org, industry, concept)={
    edge_index=[2, 98],
    edge_label=[36],
    edge_label_index=[2, 36],
  },
  (concept, subclass of, concept)={ edge_index=[2, 352] },
  (org, subsidiary, org)={ edge_index=[2, 182] },
  (loc, rev_country, org)={ edge_index=[2, 260] },
  (concept, rev_industry, org)={ edge_index=[2, 98] }
)
Test data:
HeteroData(
  org={ node_id=[385

In [19]:
test_data[("org", "industry", "concept")].edge_label_index

tensor([[351,  72,  22,  47,   2, 199, 115, 231,  48, 243, 232,  19, 163, 160,
         226, 350, 105,  41, 243,  53, 305,  62, 370, 196,  52, 224, 265,  79,
         332,  50, 200, 279, 350, 241,  46, 289],
        [338,  34,  13,  46,   0, 285,   0,  76, 334, 217, 183,  77, 258, 278,
         118, 124, 296, 265, 117, 287,   0, 354, 226, 191, 232, 232, 197, 314,
         202,  46, 195, 353, 286,  14, 275, 133]])

In [20]:
test_data[("org", "industry", "concept")].edge_label

tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

### Defining Mini-Batch Loaders

We are now ready to create a mini-batch loader that will generate subgraphs that can be used as input into our GNN.
While this step is not strictly necessary for small-scale graphs, it is absolutely necessary to apply GNNs on larger graphs that do not fit onto GPU memory otherwise.
Here, we make use of the [`loader.LinkNeighborLoader`](https://pytorch-geometric.readthedocs.io/en/latest/modules/loader.html#torch_geometric.loader.LinkNeighborLoader) which samples multiple hops from both ends of a link and creates a subgraph from it.
Here, `edge_label_index` serves as the "seed links" to start sampling from.

In [21]:
from torch_geometric.loader import LinkNeighborLoader

In [22]:
# In the first hop, we sample at most 10 neighbors.
# In the second hop, we sample at most 5 neighbors.
# In addition, during training, we want to sample negative edges on-the-fly with
# a ratio of 2:1.
# We can make use of the `loader.LinkNeighborLoader` from PyG:

# Define seed edges:
edge_label_index = train_data[target_edge_type].edge_label_index
edge_label = train_data[target_edge_type].edge_label

train_loader = LinkNeighborLoader(
    data=train_data,
    num_neighbors=[10, 5],
    neg_sampling_ratio=2.0,
    edge_label_index=(target_edge_type, edge_label_index),
    edge_label=edge_label,
    batch_size=8,
    shuffle=True,
)

# Inspect a sample:
sampled_data = next(iter(train_loader))

print("Sampled mini-batch:")
print("===================")
print(sampled_data)

Sampled mini-batch:
HeteroData(
  org={
    node_id=[37],
    n_id=[37],
    num_sampled_nodes=[3],
  },
  loc={
    node_id=[2],
    n_id=[2],
    num_sampled_nodes=[3],
  },
  concept={
    node_id=[50],
    n_id=[50],
    num_sampled_nodes=[3],
  },
  (org, country, loc)={
    edge_index=[2, 10],
    e_id=[10],
    num_sampled_edges=[2],
  },
  (org, industry, concept)={
    edge_index=[2, 2],
    edge_label=[24],
    edge_label_index=[2, 24],
    e_id=[2],
    num_sampled_edges=[2],
    input_id=[8],
  },
  (concept, subclass of, concept)={
    edge_index=[2, 41],
    e_id=[41],
    num_sampled_edges=[2],
  },
  (org, subsidiary, org)={
    edge_index=[2, 12],
    e_id=[12],
    num_sampled_edges=[2],
  },
  (loc, rev_country, org)={
    edge_index=[2, 12],
    e_id=[12],
    num_sampled_edges=[2],
  },
  (concept, rev_industry, org)={
    edge_index=[2, 2],
    e_id=[2],
    num_sampled_edges=[2],
  }
)


### Creating a Heterogeneous Link-Level GNN

We are now ready to create our heterogeneous GNN.
The GNN is responsible for learning enriched node representations from the surrounding subgraphs, which can be then used to derive edge-level predictions.
For defining our heterogenous GNN, we make use of [`nn.SAGEConv`](https://pytorch-geometric.readthedocs.io/en/latest/generated/torch_geometric.nn.conv.SAGEConv.html#torch_geometric.nn.conv.SAGEConv) and the [`nn.to_hetero()`](https://pytorch-geometric.readthedocs.io/en/latest/modules/nn.html#torch_geometric.nn.to_hetero_transformer.to_hetero) function, which transforms a GNN defined on homogeneous graphs to be applied on heterogeneous ones.

In addition, we define a final link-level classifier, which simply takes both node embeddings of the link we are trying to predict, and applies a dot-product on them.

As nodes do not have any node-level information, we choose to learn their features jointly via a `torch.nn.Embedding` layer. In order to improve the expressiveness each node type, we add embedding layers for each type separately.

In [23]:
from torch_geometric.nn import SAGEConv, to_hetero
import torch.nn.functional as F

class GNN(torch.nn.Module):
    def __init__(self, hidden_channels):
        super().__init__()

        self.conv1 = SAGEConv(hidden_channels, hidden_channels)  # add two layers of SageConv
        self.conv2 = SAGEConv(hidden_channels, hidden_channels)

    def forward(self, x: torch.Tensor, edge_index: torch.Tensor) -> torch.Tensor:
        x = F.relu(self.conv1(x, edge_index))  # forward pass and apply non-linearity
        x = F.relu(self.conv2(x, edge_index))
        return x

# Our final classifier applies the dot-product between source and destination
# node embeddings to derive edge-level predictions:
class Classifier(torch.nn.Module):
    def forward(self, x_head: torch.Tensor, x_tail: torch.Tensor, edge_label_index: torch.Tensor) -> torch.Tensor:
        # Convert node embeddings to edge-level representations:
        edge_feat_head = x_head[edge_label_index[0]]
        edge_feat_tail = x_tail[edge_label_index[1]]

        # Apply dot-product to get a prediction per supervision edge:
        return (edge_feat_head * edge_feat_tail).sum(dim=-1)


class Model(torch.nn.Module):
    def __init__(self, hidden_channels):
        super().__init__()
        # Since the dataset does not come with rich features, we also learn one
        # embedding matrix for each node type ('org', 'loc', and 'concept'):
        self.embeddings_layers = torch.nn.ModuleDict()
        for entity_type in id_to_node_mapping:
          self.embeddings_layers[entity_type] = torch.nn.Embedding(data[entity_type].num_nodes, hidden_channels)

        # Instantiate homogeneous GNN:
        self.gnn = GNN(hidden_channels)

        # Convert GNN model into a heterogeneous variant:
        self.gnn = to_hetero(self.gnn, metadata=data.metadata())

        self.classifier = Classifier()

    def forward(self, data: HeteroData) -> torch.Tensor:
        x_dict = {
          entity_type: self.embeddings_layers[entity_type](data[entity_type].node_id)
          for entity_type in id_to_node_mapping
        }

        # `x_dict` holds feature matrices of all node types
        # `edge_index_dict` holds all edge indices of all edge types
        x_dict = self.gnn(x_dict, data.edge_index_dict)

        # we only try to predict one edge type, namely ('org', 'industry', 'concept')

        pred = self.classifier(
            x_dict["org"],
            x_dict["concept"],
            data[target_edge_type].edge_label_index,
        )

        return pred

Let's create the model. We will set the hidden_channels to 16 in arbitrary fashion.

In [24]:
model = Model(hidden_channels=32)

print(model)

Model(
  (embeddings_layers): ModuleDict(
    (org): Embedding(385, 32)
    (loc): Embedding(35, 32)
    (concept): Embedding(361, 32)
  )
  (gnn): GraphModule(
    (conv1): ModuleDict(
      (org__country__loc): SAGEConv(32, 32, aggr=mean)
      (org__industry__concept): SAGEConv(32, 32, aggr=mean)
      (concept__subclass_of__concept): SAGEConv(32, 32, aggr=mean)
      (org__subsidiary__org): SAGEConv(32, 32, aggr=mean)
      (loc__rev_country__org): SAGEConv(32, 32, aggr=mean)
      (concept__rev_industry__org): SAGEConv(32, 32, aggr=mean)
    )
    (conv2): ModuleDict(
      (org__country__loc): SAGEConv(32, 32, aggr=mean)
      (org__industry__concept): SAGEConv(32, 32, aggr=mean)
      (concept__subclass_of__concept): SAGEConv(32, 32, aggr=mean)
      (org__subsidiary__org): SAGEConv(32, 32, aggr=mean)
      (loc__rev_country__org): SAGEConv(32, 32, aggr=mean)
      (concept__rev_industry__org): SAGEConv(32, 32, aggr=mean)
    )
  )
  (classifier): Classifier()
)


/home/peshevskidimitar/.venvs/graph-ml/lib/python3.10/site-packages/torch_geometric/nn/to_hetero_transformer.py:151: UserWarning: The type 'subclass of' contains invalid characters which may lead to unexpected behavior. To avoid any issues, ensure that your types only contain letters, numbers and underscores.
  self.validate()


### Training a Heterogeneous Link-Level GNN

Training our GNN is then similar to training any PyTorch model.
We move the model to the desired device, and initialize an optimizer that takes care of adjusting model parameters via stochastic gradient descent.

The training loop then iterates over our mini-batches, applies the forward computation of the model, computes the loss from ground-truth labels and obtained predictions (here we make use of binary cross entropy), and adjusts model parameters via back-propagation and stochastic gradient descent.

In [25]:
import tqdm

In [26]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Device: '{device}'")

model = model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
loss_fn = torch.nn.BCELoss()  # Use binary cross-entropy since we only predict the existence (non-existence) of a specific head-rel-tail type

for epoch in range(1, 201):
    total_loss = total_examples = 0
    for sampled_data in tqdm.tqdm(train_loader):
        optimizer.zero_grad()  # reset the optimizer gradients

        sampled_data.to(device)
        pred = model(sampled_data)
        pred = torch.nn.Sigmoid()(pred)  # convert the logits to normalized probabilistic output
        ground_truth = sampled_data[target_edge_type].edge_label  # get the ground truth for the target link
        loss = loss_fn(pred, ground_truth)  # calculate the loss from the predictions and ground truth

        print(pred)
        print(ground_truth)
        print('*************')

        loss.backward()
        optimizer.step()
        total_loss += float(loss) * pred.numel()
        total_examples += pred.numel()
    print(f"Epoch: {epoch:03d}, Loss: {total_loss / total_examples:.4f}")

Device: 'cuda'


  0%|          | 0/10 [00:00<?, ?it/s]/tmp/ipykernel_14156/4152142894.py:25: UserWarning: Converting a tensor with requires_grad=True to a scalar may lead to unexpected behavior.
Consider using tensor.detach() first. (Triggered internally at /pytorch/torch/csrc/autograd/generated/python_variable_methods.cpp:835.)
  total_loss += float(loss) * pred.numel()
 10%|█         | 1/10 [00:00<00:05,  1.60it/s]

tensor([0.7694, 0.6505, 0.8863, 0.7712, 0.8627, 0.9645, 0.6318, 0.9939, 0.9775,
        0.9928, 0.9836, 0.9422, 0.5616, 0.9469, 0.7944, 0.9793, 0.9993, 0.9996,
        0.9418, 0.8504, 0.8483, 0.9955, 0.9974, 0.8734], device='cuda:0',
       grad_fn=<SigmoidBackward0>)
tensor([1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0.], device='cuda:0')
*************
tensor([0.7984, 0.9502, 0.9232, 0.8440, 0.7893, 0.8979, 0.7287, 0.7732, 0.9951,
        0.9976, 0.9808, 0.9900, 0.9705, 0.9854, 0.7667, 0.9985, 0.9343, 0.9837,
        0.9569, 0.8507, 0.8723, 0.9571, 0.9168, 0.9919], device='cuda:0',
       grad_fn=<SigmoidBackward0>)
tensor([1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0.], device='cuda:0')
*************
tensor([0.9109, 0.9576, 0.9424, 0.6023, 0.7812, 0.7768, 0.7603, 0.7999, 0.8126,
        0.9770, 0.6040, 0.8006, 0.7623, 0.9346, 0.8707, 0.9439, 0.7627, 0.9612,
        0.9935

100%|██████████| 10/10 [00:00<00:00, 12.63it/s]


tensor([1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0.], device='cuda:0')
*************
tensor([0.7688, 0.9450, 0.8616, 0.5755, 0.8473, 0.9985, 0.9777, 0.7849, 0.8154,
        0.9877, 0.9452, 0.9718, 0.9562, 0.6752, 0.8202, 0.9570, 0.8747, 0.8152,
        0.9994, 0.9483, 0.8217, 0.7846, 0.9928, 0.9375], device='cuda:0',
       grad_fn=<SigmoidBackward0>)
tensor([1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0.], device='cuda:0')
*************
tensor([0.9805, 0.7694, 0.9292, 0.9995, 0.9784, 0.7629, 0.9920, 0.7829, 0.9986,
        0.5686, 0.9882, 0.9290, 0.9732, 0.8929, 0.7985, 0.7793, 0.9737, 0.9859,
        0.9625, 0.9413, 0.8830, 0.8190, 0.8935, 0.9984], device='cuda:0',
       grad_fn=<SigmoidBackward0>)
tensor([1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0.], device='cuda:0')
*************
tensor([0.7340, 0.5924, 0.8135

...

tensor([0.8128, 0.9399, 0.6402, 0.6403, 0.9624, 0.8753, 0.8390, 0.8823, 0.5000,
        0.8504, 0.5000, 0.5000, 0.5082, 0.5001, 0.5000, 0.5000, 0.5000, 0.5003,
        0.5000, 0.5000, 0.5652, 0.5000, 0.5000, 0.5000], device='cuda:0',
       grad_fn=<SigmoidBackward0>)
tensor([1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0.], device='cuda:0')
*************
tensor([0.9439, 0.9430, 0.6871, 0.9701, 0.9702, 0.9770, 0.6099, 0.5009, 0.5000,
        0.6611, 0.5000, 0.5047, 0.5000, 0.5000, 0.5000, 0.5000, 0.5272, 0.5095,
        0.5000, 0.5081, 0.5000, 0.5691, 0.5000, 0.5000], device='cuda:0',
       grad_fn=<SigmoidBackward0>)
tensor([1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0.], device='cuda:0')
*************
tensor([0.5276, 0.8320, 0.9031, 0.9233, 0.8436, 0.8670, 0.9473, 0.9016, 0.8220,
        0.5000, 0.5086, 0.5000, 0.5000, 0.5000, 0.5738, 0.5000, 0.5000, 0.5157,
        0.5000

 80%|████████  | 8/10 [00:00<00:00, 38.64it/s]

tensor([0.9528, 0.9250, 0.7671, 0.9427, 0.9027, 0.9722, 0.9140, 0.9849, 0.6932,
        0.5000, 0.5000, 0.5000, 0.5146, 0.5000, 0.5000, 0.5056, 0.5000, 0.5000,
        0.5000, 0.5000, 0.5000, 0.6580, 0.5000, 0.5000], device='cuda:0',
       grad_fn=<SigmoidBackward0>)
tensor([1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0.], device='cuda:0')
*************


100%|██████████| 10/10 [00:00<00:00, 36.00it/s]

tensor([0.9663, 0.8748, 0.8515, 0.5621, 0.8457, 0.7773, 0.9521, 0.8046, 0.5000,
        0.5000, 0.5000, 0.5203, 0.5091, 0.5000, 0.5000, 0.5042, 0.5000, 0.5000,
        0.5000, 0.5000, 0.5000, 0.5249, 0.5000, 0.5246], device='cuda:0',
       grad_fn=<SigmoidBackward0>)
tensor([1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0.], device='cuda:0')
*************
tensor([0.7620, 0.8651, 0.9047, 0.8468, 0.8977, 0.8938, 0.5000, 0.5000, 0.5000,
        0.5000, 0.5000, 0.5001, 0.5851, 0.5155, 0.5000, 0.5031, 0.5010, 0.5215],
       device='cuda:0', grad_fn=<SigmoidBackward0>)
tensor([1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       device='cuda:0')
*************
Epoch: 200, Loss: 0.5615


In [27]:
ground_truth

tensor([1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       device='cuda:0')

In [29]:
pred

tensor([0.7620, 0.8651, 0.9047, 0.8468, 0.8977, 0.8938, 0.5000, 0.5000, 0.5000,
        0.5000, 0.5000, 0.5001, 0.5851, 0.5155, 0.5000, 0.5031, 0.5010, 0.5215],
       device='cuda:0', grad_fn=<SigmoidBackward0>)

### Evaluating a Heterogeneous Link-Level GNN

After training, we evaluate our model on useen data coming from the validation set.
For this, we define a new `LinkNeighborLoader` (which now iterates over the edges in the validation set), obtain the predictions on validation edges by running the model, and finally evaluate the performance of the model by computing the AUC score over the set of predictions and their corresponding ground-truth edges (including both positive and negative edges).

In [30]:
# Define the validation seed edges:
edge_label_index = val_data[target_edge_type].edge_label_index
edge_label = val_data[target_edge_type].edge_label

val_loader = LinkNeighborLoader(
    data=val_data,
    num_neighbors=[10, 5],
    edge_label_index=(target_edge_type, edge_label_index),
    edge_label=edge_label,
    batch_size=8,
    shuffle=False,
)

sampled_data = next(iter(val_loader))

print("Sampled mini-batch:")
print("===================")
print(sampled_data)

Sampled mini-batch:
HeteroData(
  org={
    node_id=[20],
    n_id=[20],
    num_sampled_nodes=[3],
  },
  loc={
    node_id=[2],
    n_id=[2],
    num_sampled_nodes=[3],
  },
  concept={
    node_id=[12],
    n_id=[12],
    num_sampled_nodes=[3],
  },
  (org, country, loc)={
    edge_index=[2, 10],
    e_id=[10],
    num_sampled_edges=[2],
  },
  (org, industry, concept)={
    edge_index=[2, 5],
    edge_label=[8],
    edge_label_index=[2, 8],
    e_id=[5],
    num_sampled_edges=[2],
    input_id=[8],
  },
  (concept, subclass of, concept)={
    edge_index=[2, 0],
    e_id=[0],
    num_sampled_edges=[2],
  },
  (org, subsidiary, org)={
    edge_index=[2, 1],
    e_id=[1],
    num_sampled_edges=[2],
  },
  (loc, rev_country, org)={
    edge_index=[2, 4],
    e_id=[4],
    num_sampled_edges=[2],
  },
  (concept, rev_industry, org)={
    edge_index=[2, 6],
    e_id=[6],
    num_sampled_edges=[2],
  }
)


In [31]:
from sklearn.metrics import roc_auc_score

preds = []
ground_truths = []
edges = []
for sampled_data in tqdm.tqdm(val_loader):
    with torch.no_grad():  # we set this to not calculate gradients
        edges.append(sampled_data.edge_index_dict[target_edge_type])
        sampled_data.to(device)
        out = model(sampled_data)
        p = torch.sigmoid(model(sampled_data))
        preds.append(p)
        ground_truths.append(sampled_data[target_edge_type].edge_label)

100%|██████████| 5/5 [00:00<00:00, 71.14it/s]


In [32]:
pred = torch.cat(preds, dim=0).cpu().numpy()
ground_truth = torch.cat(ground_truths, dim=0).cpu().numpy()
auc = roc_auc_score(ground_truth, pred)
print()
print(f"Validation AUC: {auc:.4f}")


Validation AUC: 0.8368


In [33]:
pred

array([0.50615543, 0.50685894, 0.50157225, 0.59955806, 0.5       ,
       0.5660353 , 0.52588826, 0.5700454 , 0.50925887, 0.5597508 ,
       0.5536032 , 0.5       , 0.5       , 0.5       , 0.5       ,
       0.5       , 0.50127804, 0.5       , 0.5       , 0.5045964 ,
       0.5       , 0.5062701 , 0.5       , 0.5       , 0.5       ,
       0.5       , 0.5       , 0.5       , 0.50111115, 0.5       ,
       0.5       , 0.5       , 0.5348674 , 0.5226287 , 0.5       ,
       0.5106532 ], dtype=float32)

In [34]:
pred.round()

array([1., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 1.,
       0., 0., 1., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 1., 1.,
       0., 1.], dtype=float32)

In [35]:
ground_truth

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0.], dtype=float32)